In [1]:
import sys
import os

#Use if working on Colab
#from google.colab import drive
#drive.mount('/content/drive')
#PATH = '/content/drive/My Drive/PPM_Stability/'

#If working locally
PATH = os.getcwd()
sys.path.append(PATH)

In [2]:
#!pip install lime==0.2.0.1
#!pip install shap==0.37.0
#!pip install xgboost==1.0.0
#!pip install anchor-exp==0.0.2.0

In [3]:
#import EncoderFactory
#from DatasetManager_for_colab import DatasetManager
#from DatasetManager import DatasetManager
#import BucketFactory
import stability as st #Nogueira, Sechidis, Brown.

import pandas as pd
import numpy as np
from scipy import stats

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler

import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict
import random
import joblib
import json

#from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC

import lime
import lime.lime_tabular
from lime import submodular_pick

#from alibi.utils.data import gen_category_map

from tqdm import tqdm_notebook

import shap

import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline

# def imp_df(column_names, importances):
#         df = pd.DataFrame({'feature': column_names,
#                        'feature_importance': importances}) \
#            .sort_values('feature_importance', ascending = False) \
#            .reset_index(drop = True)
#         return df

# # plotting a feature importance dataframe (horizontal barchart)
# def var_imp_plot(imp_df, title, num_feat):
#         imp_df.columns = ['feature', 'feature_importance']
#         b= sns.barplot(x = 'feature_importance', y = 'feature', data = imp_df.head(num_feat), orient = 'h', palette="Blues_r")

In [5]:
from lime import submodular_pick

def generate_lime_explanations(explainer,test_xi, cls, submod=False, test_all_data=None, max_feat = 10, scaler=None):
    
    #print("Actual value ", test_y)
    
#     print(type(test_xi))
#     print(type(cls.predict_proba))
#     print(type(max_feat))

    def scale_predict_fn(X):
        scaled_data = scaler.transform(X)
        pred = cls.predict_proba(scaled_data)
        return pred

    def predict_fn(X):
        #X = X.reshape(1, -1)
        pred = cls.predict_proba(X)
        return pred

    if scaler == None:
        exp = explainer.explain_instance(test_xi, 
                                 predict_fn, num_features=max_feat, labels=[0,1])
    else:
        exp = explainer.explain_instance(test_xi, 
                                 scale_predict_fn, num_features=max_feat, labels=[0,1])
        
    return exp
        
    if submod==True:
        sp_obj=submodular_pick.SubmodularPick(explainer, test_all_data, cls.predict_proba, 
                                      sample_size=20, num_features=num_features,num_exps_desired=4)
        [exp.as_pyplot_figure(label=exp.available_labels()[0]) for exp in sp_obj.sp_explanations];

In [6]:
def dispersal(weights, features):
    
    feat_len = len(features)
    weights_by_feat = []
    
    #Weights are sorted by iteration. Transpose list.
    for i in list(range(feat_len)):
        feat_weight = []
        for iteration in weights:
            feat_weight.append(iteration[i])
        weights_by_feat.append(feat_weight)
    
    dispersal = []
    dispersal_no_outlier = []
    
    for each in weights_by_feat:
        #Find mean and variance of weight for each feature
        mean = np.mean(each)
        std_dev = np.std(each)
        var = std_dev**2
        
        #Calculate relative variance, ignore features where the weight is always 0
        if mean == 0:
            dispersal.append(0)
            dispersal_no_outlier.append(0)
        else:
            rel_var = var/abs(mean)
            dispersal.append(rel_var)
            
            #dispersal without outliers - remove anything with a z-score higher
            #than 3 (more than 3 standard deviations away from the mean)
            rem_outlier = []
            z_scores = stats.zscore(each)

            for i in range(len(z_scores)):
                if -3 < z_scores[i] < 3:
                    rem_outlier.append(each[i])
            if rem_outlier != []:
                new_mean = np.mean(rem_outlier)
                if new_mean == 0:
                    dispersal_no_outlier.append(0)
                else:
                    new_std = np.std(rem_outlier)
                    new_var = new_std**2
                    new_rel_var = new_var/abs(new_mean)
                    dispersal_no_outlier.append(new_rel_var)
            else:
                dispersal_no_outlier.append(rel_var)

    return dispersal, dispersal_no_outlier

In [7]:
def create_samples(shap_explainer, iterations, row, features, pred, top = None, scaler = None):
    length = len(features)
    
    exp = []
    rel_exp = []
    
    if scaler != None:
        row = scaler.transform(row)
    
    for j in range(iterations):
        #Generate shap values for row
        if type(shap_explainer) == shap.explainers._tree.Tree:
            shap_values = shap_explainer(row, check_additivity = False).values
        else:
            shap_values = shap_explainer(row.reshape(1, -1)).values
        
        #print(exp.shape)
        #print(exp)
        #print(shap_values.shape)
        #print(len(features))
        if shap_values.shape == (1, len(features), 2):
            shap_values = shap_values[0]
            
        #print(exp.shape)
        
        if shap_values.shape == (len(features), 2):
            shap_values = np.array([feat[pred] for feat in shap_values]).reshape(len(features))
        elif shap_values.shape == (1, len(row)) or shap_values.shape == (len(features), 1):
            shap_values = shap_values.reshape(len(features))
            
        #print(np.array(exp).shape)
        
        if scaler != None:
            #print(shap_values)
            shap_values = scaler.inverse_transform(shap_values.reshape(1, -1))[0]
            #print(shap_values.shape)
        
        #Map SHAP values to feature names
        importances = []
        
        abs_values = []
    
        for i in range(length):
            feat = features[i]
            shap_val = shap_values[i]
            abs_val = abs(shap_values[i])
            entry = (feat, shap_val, abs_val)
            importances.append(entry)
            abs_values.append(abs_val)
        
        #Sort features by influence on result
        importances.sort(key=lambda tup: tup[2], reverse = True)
        
        #Create list of all feature
        exp.append(importances)
        
        #print(exp[0])
        
        #Create list of most important features
        rel_feat = []
        if top != None:
            for i in range(top):
                feat = importances[i]
                if feat[2] > 0:
                    rel_feat.append(feat)

            rel_exp.append(rel_feat)
        else:
            bins = pd.cut(abs_values, 4, duplicates = "drop", retbins = True)[-1]
            q1_min = bins[-2]
            rel_feat = [feat for feat in importances if feat[2] > q1_min]
            rel_exp.append(rel_feat)
        
    return exp, rel_exp

In [8]:
dataset_ref = "nursery"
cls_method = "logit"

xai_method = "SHAP"

sample_size = 2
exp_iter = 5
max_feat = 10

In [9]:
# for dataset_name in datasets:

#     min_prefix_length = 1

#     dataset_manager = DatasetManager(dataset_name)
#     data = dataset_manager.read_dataset()

#     all_pipelines = []
#     all_cls = []
#     all_encoders = []
#     all_scalers = []
#     all_train = []
#     all_samples = []
#     all_results = []
    
#     for ii in range(n_iter):
#         num_buckets = len([name for name in os.listdir(os.path.join(PATH,'%s/%s/%s/pipelines'% (dataset_ref, cls_method, method_name)))])

#         for bucket in tqdm_notebook(range(num_buckets)):
#             bucketID = bucket+1
#             print ('Bucket', bucketID)

#             #import everything needed to sort and predict
#             pipeline_path = os.path.join(PATH, "%s/%s/%s/pipelines/pipeline_bucket_%s.joblib" % 
#                                          (dataset_ref, cls_method, method_name, bucketID))
#             pipeline = joblib.load(pipeline_path)
#             feature_combiner = pipeline['encoder']
#             if 'scaler' in pipeline.named_steps:
#                 scaler = pipeline['scaler']
#             else:
#                 scaler = None
#             cls = pipeline['cls']
            
#             all_cls.append(cls)
#             all_encoders.append(feature_combiner)
#             all_scalers.append(scaler)
#             all_pipelines.append(pipeline)

#             #find relevant samples for bucket
#             bucket_sample = pd.read_csv(os.path.join(PATH, "%s/%s/%s/samples/test_sample_bucket_%s.csv" % 
#                                       (dataset_ref, cls_method, method_name, bucketID))).values
#             results_template = pd.read_csv(os.path.join(PATH, "%s/%s/%s/samples/results_bucket_%s.csv" % 
#                                       (dataset_ref, cls_method, method_name, bucketID)))
    
#             if scaler != None:
#                 bucket_sample = scaler.transform(bucket_sample)
#             bucket_results = results_template
            
#             feat_names = feature_combiner.get_feature_names()
#             feat_list = [feat.replace(" ", "_") for feat in feat_names]
            
#             all_samples.append(bucket_sample)
#             all_results.append(bucket_results)
            
#             #import training data for bucket
#             train_data = pd.read_csv(os.path.join(PATH, "%s/%s/%s/train_data/train_data_bucket_%s.csv" % 
#                                                           (dataset_ref, cls_method, method_name, bucketID))).values
#             if scaler != None:
#                 train_data = scaler.transform(train_data)
            
#             all_train.append(train_data)

dataset_path = os.path.join(PATH, dataset_ref)

cls = joblib.load(os.path.join(dataset_path, cls_method, "cls.joblib"))
scaler = joblib.load(os.path.join(dataset_path, "scaler.joblib"))

trainingdata = pd.read_csv(os.path.join(dataset_path, "datasets", dataset_ref+"_Xtrain.csv"), sep=";")
with open(os.path.join(dataset_path, "datasets",'col_dict.json')) as file:
    col_dict = json.load(file)
file.close()

sample_instances = pd.read_csv((os.path.join(dataset_path, cls_method, "test_sample.csv")), sep=";") 
results = pd.read_csv((os.path.join(dataset_path, cls_method, "results.csv")), sep=";") 
col_dict

{'continuous': None,
 'discrete': ['parents_great_pret',
  'parents_pretentious',
  'parents_usual',
  'has_nurs_critical',
  'has_nurs_improper',
  'has_nurs_less_proper',
  'has_nurs_proper',
  'has_nurs_very_crit',
  'form_complete',
  'form_completed',
  'form_foster',
  'form_incomplete',
  'children_1',
  'children_2',
  'children_3',
  'children_more',
  'housing_convenient',
  'housing_critical',
  'housing_less_conv',
  'finance_convenient',
  'finance_inconv',
  'social_nonprob',
  'social_problematic',
  'social_slightly_prob',
  'health_not_recom',
  'health_priority',
  'health_recommended']}

In [10]:
if xai_method=="SHAP":

    if cls_method == "xgboost" or cls_method == "decision_tree":
        shap_explainer = shap.Explainer(cls)
    elif cls_method == "nb":
        shap_explainer = shap.Explainer(cls.predict_proba, trainingdata)
    else:
        shap_explainer = shap.Explainer(cls, trainingdata)
    print(type(shap_explainer))

    feat_list = trainingdata.columns.tolist()

    subset_stability = []
    weight_stability = []
    adjusted_weight_stability = []

    #explain the chosen instances and find the stability score
    instance_no = 0
    for instance in tqdm_notebook(sample_instances.values):
        instance_no += 1    
        print("Testing", instance_no, "of", len(sample_instances), ".")

        #if cls_method == "xgboost":
        instance = instance.reshape(1, -1)
        pred = cls.predict(instance)

        #Get Tree SHAP explanations for instance
        exp, rel_exp = create_samples(shap_explainer, exp_iter, instance, feat_list, pred, scaler = scaler)

        feat_pres = []
        feat_weights = []

        for iteration in rel_exp:
            #print("Computing feature presence for iteration", rel_exp.index(iteration))

            presence_list = [0]*len(feat_list)

            for each in feat_list:
                list_idx = feat_list.index(each)

                for explanation in iteration:
                    if each in explanation[0]:
                        presence_list[list_idx] = 1

            feat_pres.append(presence_list)

        for iteration in exp:
            #print("Compiling feature weights for iteration", exp.index(iteration))

            weights = [0]*len(feat_list)

            for each in feat_list:
                list_idx = feat_list.index(each)

                for explanation in iteration:
                    if each in explanation[0]:

                        weights[list_idx] = explanation[1]
            feat_weights.append(weights)

        stability = st.getStability(feat_pres)
        print ("Stability:", round(stability,2))
        subset_stability.append(stability)

        rel_var, second_var = dispersal(feat_weights, feat_list)
        avg_dispersal = 1-np.mean(rel_var)
        print ("Dispersal of feature importance:", round(avg_dispersal, 2))
        weight_stability.append(avg_dispersal)
        adj_dispersal = 1-np.mean(second_var)
        print ("Dispersal with no outliers:", round(adj_dispersal, 2))
        adjusted_weight_stability.append(adj_dispersal)

    results["SHAP Subset Stability"] = subset_stability
    results["SHAP Weight Stability"] = weight_stability
    results["SHAP Adjusted Weight Stability"] = adjusted_weight_stability


<class 'shap.explainers._linear.Linear'>


Testing 1 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 2 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 3 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 4 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 5 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 6 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 7 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 8 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 9 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 10 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
D

Testing 83 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 84 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 85 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 86 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 87 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 88 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 89 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 90 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 91 of 100 .
Stability: 1.0
Dispersal of feature importance: 1.0
Dispersal with no outliers: 1.0
Testing 92 of 100 .
Stability: 1.0
Dispersal of feature importan

In [11]:
results

,Actual,Prediction,Prediction Probability,SHAP Subset Stability,SHAP Weight Stability,SHAP Adjusted Weight Stability
0,0,0,0.997319,1.0,1.0,1.0
1,0,0,0.997458,1.0,1.0,1.0
2,0,0,0.997196,1.0,1.0,1.0
3,0,0,0.997522,1.0,1.0,1.0
4,0,0,0.997529,1.0,1.0,1.0
...,...,...,...,...,...,...
95,1,1,0.997351,1.0,1.0,1.0
96,1,1,0.997485,1.0,1.0,1.0
97,1,1,0.997405,1.0,1.0,1.0
98,1,1,0.997315,1.0,1.0,1.0


In [12]:
results.to_csv(os.path.join(dataset_path, cls_method, "results.csv"))

In [13]:
if xai_method=="LIME":
    
    feat_list = trainingdata.columns.tolist()
    class_names = ["Negative", "Positive"]

    cats = col_dict["discrete"]

    subset_stability = []
    weight_stability = []
    adjusted_weight_stability = []

    #create explainer now that can be passed later
    lime_explainer = lime.lime_tabular.LimeTabularExplainer(trainingdata.values,
                          feature_names = feat_list, class_names=class_names, categorical_features = cats)

    instance_no = 0
    print(len(sample_instances))
    #explain the chosen instances and find the stability score
    for instance in tqdm_notebook(sample_instances.values):
        instance_no += 1

        print("Testing", instance_no, "of", len(sample_instances), ".")

        #Get lime explanations for instance
        feat_pres = []
        feat_weights = []

        for iteration in list(range(exp_iter)):

            lime_exp = generate_lime_explanations(lime_explainer, instance, cls,
                                                  max_feat = len(feat_list), scaler = scaler)

            all_weights = [exp[1] for exp in lime_exp.as_list()]
            bins = pd.cut(all_weights, 4, duplicates = "drop", retbins = True)[-1]
            q1_min = bins[-2]

            presence_list = [0]*len(feat_list)
            weights = [0]*len(feat_list)

            for each in feat_list:
                list_idx = feat_list.index(each)
                #print ("Feature", list_idx)
                for explanation in lime_exp.as_list():
                    if each in explanation[0]:
                        if explanation[1] > q1_min:
                            presence_list[list_idx] = 1
                        weights[list_idx] = explanation[1]

            feat_pres.append(presence_list)
            feat_weights.append(weights)

        stability = st.getStability(feat_pres)
        print ("Stability:", round(stability,2))
        subset_stability.append(stability)

        rel_var, second_var = dispersal(feat_weights, feat_list)
        avg_dispersal = 1-np.mean(rel_var)
        print ("Dispersal of feature importance:", round(avg_dispersal, 2))
        weight_stability.append(avg_dispersal)
        adj_dispersal = 1-np.mean(second_var)
        print ("Dispersal with no outliers:", round(adj_dispersal, 2))
        adjusted_weight_stability.append(adj_dispersal)

    results["LIME Subset Stability"] = subset_stability
    results["LIME Weight Stability"] = weight_stability
    results["LIME Adjusted Weight Stability"] = adjusted_weight_stability

In [14]:
results

,Actual,Prediction,Prediction Probability,SHAP Subset Stability,SHAP Weight Stability,SHAP Adjusted Weight Stability
0,0,0,0.997319,1.0,1.0,1.0
1,0,0,0.997458,1.0,1.0,1.0
2,0,0,0.997196,1.0,1.0,1.0
3,0,0,0.997522,1.0,1.0,1.0
4,0,0,0.997529,1.0,1.0,1.0
...,...,...,...,...,...,...
95,1,1,0.997351,1.0,1.0,1.0
96,1,1,0.997485,1.0,1.0,1.0
97,1,1,0.997405,1.0,1.0,1.0
98,1,1,0.997315,1.0,1.0,1.0


In [15]:
results.to_csv(os.path.join(dataset_path, cls_method, "results.csv"))

In [16]:
testingdata = all_samples[0]
instance = all_samples[0][0]
trainingdata = all_train[0]

cls = all_cls[0]
scaler = all_scalers[0]

feat_list = [feat.replace(" ", "_") for feat in feature_combiner.get_feature_names()]

class_names = ["Negative", "Positive"]

NameError: name 'all_samples' is not defined

In [ ]:
lime_explainer = lime.lime_tabular.LimeTabularExplainer(trainingdata,
                                  feature_names = feat_list, class_names=class_names)
if scaler == None:
    exp = lime_explainer.explain_instance(instance, 
                                 cls.predict_proba, num_features=10, labels=[0,1], top_labels=1)
else:
    exp = lime_explainer.explain_instance(instance, 
                             scale_predict_fn, num_features=max_feat, labels=[0,1])

In [ ]:
exp.as_pyplot_figure(0)

In [ ]:
!pip show joblib

In [ ]:
exp.show_in_notebook()

In [ ]:
shap_explainer = shap.Explainer(cls)
explanation = shap_explainer(testingdata)
explanation.feature_names = feat_list
shap.plots.waterfall(explanation[0])

In [ ]:
trainingdata.shape

In [ ]:
np.unique(scaler.inverse_transform(trainingdata), axis=0)

In [ ]:
dataset_manager = DatasetManager(dataset_name)
dataset_manager.static_cat_cols

In [ ]:
dataset_manager.dynamic_cat_cols

In [ ]:
dataset_manager.static_num_cols

In [ ]:
cats = [feat for col in dataset_manager.dynamic_cat_cols+dataset_manager.static_cat_cols for feat in range(len(feat_list)) if col in feat_list[feat]]

In [ ]:
np.array(feat_list)[cats]